##### File containing the vessel definitions

Script modified from the work by Leonore Vehmeijer. Changes:
- Changed fieldnames (more consistent naming
- Added more class information (avv_class, cemt_class, type)

Sources:
- RWS, 2002. "Classificatie en kernmerken van de Europese vloot en de Actieve vloot in Nederland" Rijkswaterstaat, Adviesdienst Verkeer en Vervoer. 
- https://nl.wikipedia.org/wiki/CEMT-klasse

**Todo:**
- Thoroughly check the list and document where values came from
- Reconsider whether the loaded option should be included (doesn't seem very useful)
- Include 'velocity_loaded' en 'velocity_unloaded'


In [1]:
import pandas as pd
import numpy as np

In [2]:
#   Bronnen
#RV2017 = Richtlijnen Vaarwegen 2017
#RWS2002 = 'Classificatie en kenmerken van de Europese vloot en de Actieve vloot in Nederland'
#Marin2010 = Scheepskarakteristieken van nieuwe grote schepen, 2010

#   Gemiddelde lengte en breedte
#Voor (bijna) alle lengte en breedte maten van schepen heb ik nu een gemiddelde waarde uit RWS2002 genomen
#De waardes die Leonore gebruikte zijn de maatgevende maten van iedere vessel class (dus groter dan gemiddeld)
#Naar mijn idee kunnen we beter het gemiddelde gebruiken
#Er is in het RWS2002 document ook een st.dev. gegeven van o.a. de gemiddelde L en B
#Wellicht kunnen we L en B inbrengen als stochastische variabele, met een normaalverdeling met deze st.dev

#   T_e en T_f
#Alleen voor de M1-M12 schepen is expliciet een T_e en T_f genoemd in Tabel 2 in RV2017. 
#Voor de andere scheepstypen is er alleen een T_f genoemd, in Tabel 8 in RV2017.
#Daarom is op basis van de verhouding T_f / T_e van M1-M12 schepen (= 1.9 gemiddeld) de T_e voor andere boottypen berekend

#Is_loaded
#Betekent de Is_loaded attribute dat een schip alleen volledig geladen, of volledig ongeladen is? En is dat realistisch?

M1 = {
    'B': 5.06,                                     # RWS2002 Gemiddelde B             
    'L':39.76,                                     # RWS2002 Gemiddelde L
    'H_e': 2,                                      # ??    Moet dit niet H_f zijn      Misschien is dit H_f - (T_f - T_e)
    'H_f': 5.25,                                   # RV2017 Tabel 8 - Maatgevende Doorvaarthoogte, incl 30 cm schrikhoogte
    'T_e': 1.2,                                    # RV2017 Tabel 2      
    'T_f': 2.5,                                    # RV2017 Tabel 2
    'emissionfactor': 0.73,
    'P_installed': 36,
    'own_weight': 80,
    'capacity_loaded':400*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.083,
    'speed_unloaded': 4.47,
    'resistance_loaded': 16919,
    'resistance_unloaded':9188,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M1",                             # RV2017 Tabel 8
    'cemt_class': "I",                             # RV2017 Tabel 8
    'type': "Spits",                               # RV17 Tabel 8
}

M2 = {
    'B': 6.42,                                     # RWS2002 Gemiddelde B
    'L':52.26,                                     # RWS2002 Gemiddelde L
    'H_e': 2,                                      # ?? Iedere H_e (wat denk ik H_f moet zijn) is gesteld op de waarde 2
    'H_f': 6.1,                                    # RV2017 Tabel 8
    'T_e': 1.4,                                    # RV2017 Tabel 2
    'T_f': 2.6,                                    # RV2017 Tabel 2
    'emissionfactor': 0.72,
    'P_installed': 118,
    'own_weight': 90,
    'capacity_loaded':650*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.361,
    'speed_unloaded': 4.638,
    'resistance_loaded': 28770,
    'resistance_unloaded':17524,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M2",                             # RV17 Tabel 8
    'cemt_class': "II",                            # RV17 Tabel 8
    'type': "Kempenaar",                           # RV17 Tabel 8
}

M3 = {
    'B': 7.15,                                     # RWS2002 Gemiddelde B
    'L':72.21,                                     # RWS2002 Gemiddelde L
    'H_e': 2,                                      # ?? 
    'H_f': 6.4,                                    # RV2017 Tabel 8
    'T_e': 1.5,                                    # RV2017 Tabel 2
    'T_f': 2.6,                                    # RV2017 Tabel 2
    'emissionfactor': 0.72,
    'P_installed': 120,
    'own_weight': 100,
    'capacity_loaded':800*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.861,
    'speed_unloaded': 4.638,
    'resistance_loaded': 43935,
    'resistance_unloaded':22746,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M3",                             # RV2017
    'cemt_class': "III",                           # RV2017
    'type': "Hagenaar",                            # RV2017
}

M4 = {
    'B': 8.02,                                     # RWS2002 Gemiddelde B
    'L':67.02,                                     # RWS2002 Gemiddelde L
    'H_e': 2,                                      # ?? 
    'H_f': 6.6,                                    # RV2017 Tabel 8
    'T_e': 1.5,                                    # RV2017 Tabel 2
    'T_f': 2.7,                                    # RV2017 Tabel 2
    'emissionfactor': 0.72,
    'P_installed': 160,
    'own_weight': 110,
    'capacity_loaded':1050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 4.000,
    'speed_unloaded': 4.889,
    'resistance_loaded': 50582,
    'resistance_unloaded':28777,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M4",                             # RV2017
    'cemt_class': "III",                           # RV2017
    'type': "Dortmund-Eems (L <= 74 m)",           # RV2017
}

M5 = {
    'B': 8.18,                                     # RWS2002 Gemiddelde B
    'L':80.56,                                     # RWS2002 Gemiddelde L
    'H_e': 2,                                      # ??
    'H_f': 6.4,                                    # RV2017 Tabel 8
    'T_e': 1.5,                                    # RV2017 Tabel 2
    'T_f': 2.7,                                    # RV2017 Tabel 2
    'emissionfactor': 0.72,
    'P_installed': 323,
    'own_weight': 140,
    'capacity_loaded':1250*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.916,
    'speed_unloaded': 5.027,
    'resistance_loaded': 54096,
    'resistance_unloaded':31794,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M5",                             # RV2017
    'cemt_class': "III",                           # RV2017
    'type': "Verl. Dortmund (L > 74 m)",           # RV2017
}

M6 = {
    'B': 9.23,                                     # RWS2002 Gemiddelde B
    'L':80.56,                                     # RWS2002 Gemiddelde L
    'H_e': 2,                                      # ??
    'H_f': 7,                                      # RV2017 Tabel 8
    'T_e': 1.6,                                    # RV2017 Tabel 2
    'T_f': 2.9,                                    # RV2017 Tabel 2
    'emissionfactor': 0.71,
    'P_installed': 263,
    'own_weight': 120,
    'capacity_loaded':1750*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.916,
    'speed_unloaded': 5.194,
    'resistance_loaded': 70802,
    'resistance_unloaded':44676,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M6",                             # RV2017 Tabel 2
    'cemt_class': "IV",                            # RV2017 Tabel 2
    'type': "Rijn-Herne Schip (L <= 86 m)",        # RV2017 Tabel 2
}

M7 = {
    'B': 9.41,                                     # RWS2002 Gemiddelde B
    'L':100.87,                                    # RWS2002 Gemiddelde L
    'H_e': 2,                                      # ??
    'H_f': 7,                                      # RV2017 Tabel 8
    'T_e': 1.6,                                    # RV2017 Tabel 2
    'T_f': 3.0,                                    # RV2017 Tabel 2
    'emissionfactor': 0.70,
    'P_installed': 258,
    'own_weight': 110,
    'capacity_loaded':2050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.778,
    'speed_unloaded': 5.361,
    'resistance_loaded': 74294,
    'resistance_unloaded':53220,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M7",                             # RV2017
    'cemt_class': "IV",                            # RV2017
    'type': "Verl. Rijn-Herne Schip (L > 86 m)",   # RV2017
}

M8 = {
    'B': 11.32,                                    # Marin2010 Gemiddelde B
    'L': 106,                                      # Marin2010 Gemiddelde L
    'H_e': 4,                                      # ?? Vanaf hier ineens 4?
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 1.8,                                    # RV2017 Tabel 2
    'T_f': 3.5,                                    # RV2017 Tabel 2
    'emissionfactor': 0.7,
    'P_installed': 318,
    'own_weight': 100,
    'capacity_loaded':2711*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.7777,
    'speed_unloaded': 5.3333,
    'resistance_loaded': 106453,
    'resistance_unloaded':61526,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M8",                             # RV2017
    'cemt_class': "Va",                            # RV2017
    'type': "Groot Rijnschip",                     # RV2017
}

M9 = {
    'B': 11.45,                                    # Marin2010 Gemiddelde B
    'L':133,                                       # Marin2010 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 1.8,                                    # RV2017 Tabel 2
    'T_f': 3.5,                                    # RV2016 Tabel 2
    'emissionfactor': 0.70,
    'P_installed': 350,
    'own_weight': 120,
    'capacity_loaded':4000*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.805,
    'speed_unloaded': 5.416,
    'resistance_loaded': 116161,
    'resistance_unloaded':69833,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M9",                             # RV2017
    'cemt_class': "Va",                            # RV2017
    'type': "Verl. Groot Rijnschip",               # RV2017
}

M10 = {
    'B': 13.52,                                    # Marin2010 Gemiddelde B
    'L':107,                                       # Marin2010 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 2.0,                                    # RV2017 Tabel 2
    'T_f': 4.0,                                    # RV2017 Tabel 2
    'emissionfactor': 0.70,
    'P_installed': 350,
    'own_weight': 120,
    'capacity_loaded':4300*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.583,
    'speed_unloaded': 5.388,
    'resistance_loaded': 124101,
    'resistance_unloaded':72153,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M10",                            # RV2017 Tabel 8
    'cemt_class': "VIa",                           # RV2017 Tabel 8
    'type': "Maatgevend schip 13.5 * 110 m",       # RV2017 Tabel 8
}

M11 = {
    'B': 13.8,                                     # Marin2010 Gemiddelde B
    'L':134,                                       # Marin2010 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'emissionfactor': 0.70,
    'T_e': 2.0,                                    # RV2017 Tabel 2
    'T_f': 4.0,                                    # RV2017 Tabel 2
    'P_installed': 380,
    'own_weight': 150,
    'capacity_loaded':5600*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 4.027,
    'speed_unloaded': 5.277,
    'resistance_loaded': 141593,
    'resistance_unloaded':85947,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M11",                            # RV2017 Tabel 8
    'cemt_class': "VIa",                           # RV2017 Tabel 8
    'type': "Maatgevend schip 14.2 * 135 m",       # RV2017 Tabel 8
}

M12 = {
    'B': 16.43,                                    # Marin2010 Gemiddelde B
    'L':135,                                       # Marin2010 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 2.0,                                    # RV2017 Tabel 2
    'T_f': 4.0,                                    # RV2017 Tabel 2
    'emissionfactor': 0.70,
    'P_installed': 380,
    'own_weight': 145,
    'capacity_loaded':5600*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.888,
    'speed_unloaded': 5.138,
    'resistance_loaded': 167863,
    'resistance_unloaded':102290,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M12",                            # RV2017 Tabel 8
    'cemt_class': "VIa",                           # RV2017 Tabel 8
    'type': "Rijnmax schip",                       # RV2017 Tabel 8
}

BI = {
    'B': 9.46,                                     # RWS2002 Gemiddelde B
    'L':95,                                        # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 7.0,                                    # RV2017 Tabel 8
    'T_e': 1.6,                                    # T_f / 1.9 (zie notes bovenaan)
    'T_f': 3.0,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 263,
    'own_weight': 130,
    'capacity_loaded':1800*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.666,
    'speed_unloaded': 4.444,
    'resistance_loaded': 63671,
    'resistance_unloaded':27159,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BI",                             # RV2017 Tabel 8
    'cemt_class': "IV",                            # RV2017 Tabel 8
    'type': "1 baksduwstel Europa I",              # RWS2002
}

BII_1 = {
    'B': 11.4,                                     # RWS2002 Gemiddelde B
    'L':102,                                       # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 1.8,                                    # T_f / 1.9
    'T_f': 3.5,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 318,
    'own_weight': 135,
    'capacity_loaded':3950*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.555,
    'speed_unloaded': 4.778,
    'resistance_loaded': 85750,
    'resistance_unloaded':37792,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_1",                          # RWS2002
    'cemt_class': "Va",                            # RWS2002
    'type': "1 baksduwstel Europa II",             # RWS2002
}

BII_2b = {
    'B': 22.59,                                   # RWS2002 Gemiddelde B
    'L':113,                                      # RWS2002 Gemiddelde L
    'H_e': 3,                                     # ??
    'H_f': 8,                                     # RV2017 Tabel 8
    'T_e': 2.0,                                   # T_f / 1.9
    'T_f': 3.8,                                   # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 350,
    'own_weight': 150,
    'capacity_loaded':7050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.055,
    'speed_unloaded':4.722,
    'resistance_loaded': 140637,
    'resistance_unloaded':77029,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_2b",                         # RWS2002
    'cemt_class': "VIa",                           # RWS2002
    'type': "2 baksduwstel (breed)",               # RWS2002
}

BII_2l = {
    'B': 11.48,                                    # RWS2002 Gemiddelde B
    'L':180,                                       # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 2.0,                                    # T_f / 1.9
    'T_f': 3.8,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 350,
    'own_weight': 140,
    'capacity_loaded':7050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.638,
    'speed_unloaded':4.972,
    'resistance_loaded': 129810,
    'resistance_unloaded':55974,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_2l",                         # RWS2002
    'cemt_class': "Vb",                            # RWS2002
    'type': "2 baksduwstel (lang)",                # RWS2002
}

## Lower height for NS - Low scenario
BII_4 = {
    'B': 22.78,                                    # RWS2002 Gemiddelde B
    'L':189,                                       # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 2.0,                                    # T_f / 1.9
    'T_f': 3.8,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 400,
    'own_weight': 160,
    'capacity_loaded':12000*0.60,
    'capacity_unloaded':0,
    'speed_loaded': 2.833,
    'speed_unloaded':4.611,
    'resistance_loaded': 861913,
    'resistance_unloaded':114953,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_4",                          # RWS2002
    'cemt_class': "VIb",                           # RWS2002
    'type': "4 baksduwstel",                       # RWS2002
}

BII_6l = {
    'B': 22.77,                                    # RWS2002 Gemiddelde B
    'L':263,                                       # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 2.0,                                    # T_f / 1.9
    'T_f': 3.8,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 400,
    'own_weight': 165,
    'capacity_loaded':18000*0.5,
    'capacity_unloaded':0,
    'speed_loaded': 2.667,
    'speed_unloaded':4.472,
    'resistance_loaded': 213665,
    'resistance_unloaded':151739,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_6l",                         # RWS2002
    'cemt_class': "VIc",                           # RWS2002
    'type': "6 baksduwstel (lang)",                # RWS2002
}

B01 = {
    'B': 5.04,                                     # RWS2002 Gemiddelde B
    'L':53,                                        # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 5.25,                                   # RV2017 Tabel 8
    'T_e': 1.0,                                    # T_f / 1.9
    'T_f': 1.9,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 36,
    'own_weight': 80,
    'capacity_loaded':400*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.889,
    'speed_unloaded':4.167,
    'resistance_loaded': 27281,
    'resistance_unloaded':10473,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "B01",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwstel: 5.20 * 55",                  # RWS2002
}

B02 = {
    'B': 6.43,                                     # RWS2002 Gemiddelde B
    'L':62,                                        # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 6.1,                                    # RV2017 Tabel 8
    'T_e': 1.4,                                    # T_f / 1.9
    'T_f': 2.6,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 118,
    'own_weight': 90,
    'capacity_loaded':600*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.333,
    'speed_unloaded':4.305,
    'resistance_loaded': 36338,
    'resistance_unloaded':15187,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "B02",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwstel: 6.70 * 61",                  # RWS2002
}

B03 = {
    'B': 7.41,                                     # RWS2002 Gemiddelde B
    'L':74,                                        # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 6.4,                                    # RV2017 Tabel 8
    'T_e': 1.4,                                    # T_f / 1.9
    'T_f': 2.6,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 120,
    'own_weight': 95,
    'capacity_loaded':800*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.667,
    'speed_unloaded':4.278,
    'resistance_loaded': 42647,
    'resistance_unloaded':18677,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "B03",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwstel: 7.50 * 78",                  # RWS2002
}

B04 = {
    'B': 8.12,                                     # RWS2002 Gemiddelde B
    'L':78,                                        # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 6.6,                                    # RV2017 Tabel 8
    'T_e': 1.4,                                    # T_f / 1.9
    'T_f': 2.7,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 160,
    'own_weight': 95,
    'capacity_loaded':1250*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.778,
    'speed_unloaded':4.361,
    'resistance_loaded': 47147,
    'resistance_unloaded':21147,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "B04",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwstel: 8.20 * 85",                  # RWS2002
}

C1b = {
    'B': 10.28,                                     # RWS2002 Gemiddelde B
    'L':46,                                         # RWS2002 Gemiddelde L
    'H_e': 3,                                       # ??
    'H_f': 5.25,                                    # RV2017 Tabel 8
    'T_e': 1.3,                                     # T_f / 1.9
    'T_f': 2.5,                                     # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 36,
    'own_weight': 80,
    'capacity_loaded':900*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.00,
    'speed_unloaded':4.222,
    'resistance_loaded': 28204,
    'resistance_unloaded':15387,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C1b",                            # RWS2002
    'cemt_class': "I",                             # RWS2002
    'type': "2 Spitsen breed",                     # RWS2002
}

C1l = {
    'B': 5.07,                                     # RWS2002 Gemiddelde B
    'L':79,                                        # RWS2002 Gemiddelde L
    'H_e': 3,                                      # RWS2002 Gemiddelde ??
    'H_f': 5.25,                                   # RV2017 Tabel 8
    'T_e': 1.3,                                    # T_f / 1.9
    'T_f': 2.5,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 36,
    'own_weight': 80,
    'capacity_loaded':900*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.278,
    'speed_unloaded':4.389,
    'resistance_loaded': 20357,
    'resistance_unloaded':15211,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C1l",                            # RWS2002
    'cemt_class': "I",                             # RWS2002
    'type': "2 Spitsen lang",                      # RWS2002
}

C2b = {
    'B': 17.72,                                    # RWS2002 Gemiddelde B
    'L':89,                                        # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 7.0,                                    # RWS2002 Gemiddelde B
    'T_e': 1.6,                                    # T_f / 1.9
    'T_f': 3.0,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 350,
    'own_weight':140,
    'capacity_loaded':3350*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 2.583,
    'speed_unloaded':4.167,
    'resistance_loaded': 93843,
    'resistance_unloaded':70473,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C2b",                            # RWS2002
    'cemt_class': "VIa",                           # RWS2002
    'type': "Klasse IV + 1 Europa I bak breed",    # RWS2002
}

C2l = {
    'B': 9.16,                                     # RWS2002 Gemiddelde B
    'L':157,                                       # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 7.0,                                    # RV2017 Tabel 8
    'T_e': 1.6,                                    # T_f / 1.9
    'T_f': 3.0,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 258,
    'own_weight':140,
    'capacity_loaded':3350*0.60,
    'capacity_unloaded':0,
    'speed_loaded': 3.583,
    'speed_unloaded':4.722,
    'resistance_loaded': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C2l",                            # RWS2002
    'cemt_class': "IVb",                           # RWS2002
    'type': "Klasse IV + 1 Europa I bak lang",     # RWS2002
}

C3b = {
    'B': 22.46,                                    # RWS2002 Gemiddelde B
    'L':105,                                       # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 2.0,                                    # T_f / 1.9
    'T_f': 3.8,                                    # RV2016 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 350,
    'own_weight':140,
    'capacity_loaded':7250*0.5,
    'capacity_unloaded':0,
    'speed_loaded': 2.333,
    'speed_unloaded':4.278,
    'resistance_loaded': 93843,
    'resistance_unloaded':70473,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C3b",                            # RWS2002
    'cemt_class': "VIa",                           # RWS2002
    'type': "Klasse Va + 1 Europa II bak breed",   # RWS2002
}

C3l = {
    'B': 11.37,                                    # RWS2002 Gemiddelde B
    'L':178,                                       # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RV2017 Tabel 8
    'T_e': 2.0,                                    # T_f / 1.9
    'T_f': 3.8,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 350,
    'own_weight':140,
    'capacity_loaded':7250*0.60,
    'capacity_unloaded':0,
    'speed_loaded': 3.250,
    'speed_unloaded':5.056,
    'resistance_loaded': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C3l",                            # RWS2002
    'cemt_class': "Vb",                            # RWS2002
    'type': "Klasse Va + 1 Europa II bak lang",    # RWS2002
}

C4 = {
    'B': 22.38,                                    # RWS2002 Gemiddelde B
    'L':182,                                       # RWS2002 Gemiddelde L
    'H_e': 3,                                      # ??
    'H_f': 9.1,                                    # RWS2002 Gemiddelde B
    'T_e': 2.0,                                    # T_f / 1.9
    'T_f': 3.8,                                    # RV2017 Tabel 8
    'emissionfactor': 0.71,
    'P_installed': 400,
    'own_weight':150,
    'capacity_loaded':7250*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.00,
    'speed_unloaded':4.472,
    'resistance_loaded': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C4",                             # RWS2002
    'cemt_class': "VIb",                           # RWS2002
    'type': "Klasse Va + 3 Europa II bakken",      # RWS2002
}


ships = [M1, 
         M2,  
         M3,  
         M4,  
         M5,  
         M6,  
         M7,  
         M8, 
         M9,  
         M10,  
         M11,  
         M12,  
         BI,  
         BII_1,  
         BII_2l, 
         BII_2b, 
         BII_4,
         BII_6l,
         B01,  
         B02,  
         B03,  
         B04, 
         C1l,  
         C1b,  
         C2l,  
         C2b,  
         C3l,  
         C3b,  
         C4]

In [3]:
import uuid
str(uuid.uuid1())

'39681f84-e5ef-11ea-bb82-4023431303c6'

In [4]:
ship_id = ["29a24858-4a51-11e9-9792-b469212bff5b", 
           "dcc05ff4-4972-11e9-a543-b469212bff5b",  
           "e6cea50a-4972-11e9-a1f1-b469212bff5b", 
           "f0eb9740-4972-11e9-96ab-b469212bff5b",  
           "f5685128-4972-11e9-9bc8-b469212bff5b",  
           "fa18f668-4972-11e9-be98-b469212bff5b",  
           "fec15af4-4972-11e9-97be-b469212bff5b",  
           "045a2bc2-4973-11e9-ac77-b469212bff5b",  
           "0cbee714-4973-11e9-a6c3-b469212bff5b",  
           "11069770-4973-11e9-8083-b469212bff5b",  
           "147059ae-4973-11e9-8d83-b469212bff5b",  
           "1827f876-4973-11e9-84b9-b469212bff5b",  
           "1be8986e-4973-11e9-baae-b469212bff5b",  
           "24bb5f98-4973-11e9-be88-b469212bff5b",  
           "28c5310a-4973-11e9-938b-b469212bff5b",  
           "3027cb64-4973-11e9-b276-b469212bff5b",  
           "34138582-4973-11e9-9237-b469212bff5b",  
           "37eb9770-4973-11e9-87fe-b469212bff5b",  
           "3c8896ac-4973-11e9-8cff-b469212bff5b",  
           "4073635c-4973-11e9-b613-b469212bff5b",  
           "463fde30-4973-11e9-ba75-b469212bff5b", 
           "49c3124a-4973-11e9-be8c-b469212bff5b",  
           "4db2931a-4973-11e9-ae43-b469212bff5b",  
           "5200e342-4973-11e9-b821-b469212bff5b",  
           "560257d0-4973-11e9-be15-b469212bff5b",  
           "5972634c-4973-11e9-bd78-b469212bff5b",  
           "60f4dfa4-4973-11e9-9a85-b469212bff5b",  
           "667d1536-4973-11e9-a5bb-b469212bff5b", 
           "6a9cbdee-4973-11e9-84e3-b469212bff5b"]

In [5]:
vessel_db = pd.DataFrame.from_dict({'VesselID': [],
                                    'B': [],
                                    'L': [],
                                    'H_e': [],
                                    'H_f': [],
                                    'T_e': [],
                                    'T_f': [],  
                                    'emissionfactor': [],
                                    'P_installed': [],
                                    'own_weight': [],
                                    'capacity_loaded': [],
                                    'capacity_unloaded': [],
                                    'speed_loaded': [],
                                    'speed_unloaded': [],
                                    'resistance_loaded': [],
                                    'resistance_unloaded': [],
                                    'is_loaded': [],
                                    'avv_class': [],
                                    'cemt_class': [],
                                    'type': []})

for i, ship in enumerate(ships):
    ship["VesselID"] = ship_id[i]
    vessel_db = vessel_db.append(ship, ignore_index = True)

vessel_db.head()
vessel_db.to_csv("Vessel-database-2.csv", index = False)